## Data Cleaning

Empezamos por importar las librerias necesarias, configurar el espacio de trabajo y importar todos los archivos sobre los cuales vamos a trabajar.

In [186]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [187]:
from repo_funciones import * #repositorio propio de funciones
import re #para expresiones regulares

In [188]:
actor=pd.read_csv('../Data/actor.csv') 
category=pd.read_csv('../Data/category.csv')
film=pd.read_csv('../Data/film.csv')
inventory=pd.read_csv('../Data/inventory.csv')
language=pd.read_csv('../Data/language.csv')
old_HDD=pd.read_csv('../Data/old_HDD.csv')
rental=pd.read_csv('../Data/rental.csv')

## Metodología

La metodología sigue los siguientes pasos:

 - .head() para un vistazo inicial de cada tabla
 - .info() 
    - double-check dtypes
    - valores nulos
- formula propia "uniquecols". Devuelve el conteo de valores unicos por columna.
   - el conteo de unicos en "ID" de cada tabala tiene que ser igual al numero de filas, para SQL.
   - las columnas con 1 valor unico se borran, ya que no aportan información.

- duplicated().any() ver si hay duplicados. 
   - en ciertas instancias hemos ido mas allá y verificado duplicados usando subset de columnas. 


Limpieza tabla "Actor"

In [189]:
actor.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [190]:
nancols(actor) 

Series([], dtype: int64)

In [191]:
actor.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [192]:
unique_cols(actor)

actor_id       200
first_name     128
last_name      121
last_update      1
dtype: int64

In [193]:
actor = actor.drop("last_update", axis=1) #esta columna no aporta valor informativo. Solo tiene un valor unico. 

In [194]:
actor.duplicated().unique()

array([False])

In [195]:
actor.loc[actor[['first_name', 'last_name']].duplicated(keep=False)]

,actor_id,first_name,last_name
100,101,SUSAN,DAVIS
109,110,SUSAN,DAVIS


In [196]:
actor.drop_duplicates(['first_name', 'last_name'], keep=False, inplace=True)

In [197]:
actor['full_name']=actor.first_name +' '+ actor.last_name

Limpieza tabla "category"

In [198]:
category.head()

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [199]:
unique_cols(category)

category_id    16
name           16
last_update     1
dtype: int64

In [200]:
category = category.drop("last_update", axis=1) #esta columna no aporta valor informativo. Solo tiene un valor unico.  

In [201]:
nancols(category)

Series([], dtype: int64)

In [202]:
category.duplicated().unique()

array([False])

Limpieza tabla "film"

In [203]:
film.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [204]:
film.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   release_year          1000 non-null   int64  
 4   language_id           1000 non-null   int64  
 5   original_language_id  0 non-null      float64
 6   rental_duration       1000 non-null   int64  
 7   rental_rate           1000 non-null   float64
 8   length                1000 non-null   int64  
 9   replacement_cost      1000 non-null   float64
 10  rating                1000 non-null   object 
 11  special_features      1000 non-null   object 
 12  last_update           1000 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 495.3 KB


In [205]:
film = film.drop(["last_update", "original_language_id"], axis=1)

In [206]:
unique_cols(film) #intuitivamente quise borrar la columna de language_id por contener un unico valor pero luego me di cuenta que hay otra tabla de idiomas
#se mantiene ya que sera la llave que interconecte las tablas en SQL. En este caso solo hay pelis en inglés, pero queda lista para albergar varios idiomas si fuese el caso.

film_id             1000
title               1000
description         1000
length               140
replacement_cost      21
special_features      15
rental_duration        5
rating                 5
rental_rate            3
release_year           1
language_id            1
dtype: int64

In [207]:
film = film.drop("release_year", axis=1)

In [208]:
film.duplicated().unique()

array([False])

Limpieza tabla "inventory"

In [209]:
#inventory.hed()
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   inventory_id  1000 non-null   int64 
 1   film_id       1000 non-null   int64 
 2   store_id      1000 non-null   int64 
 3   last_update   1000 non-null   object
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


In [210]:
inventory = inventory.drop("last_update", axis=1)

In [211]:
unique_cols(inventory)

inventory_id    1000
film_id          207
store_id           2
dtype: int64

In [212]:
inventory.duplicated().unique()

array([False])

Limpieza tabla "language"

In [213]:
language.head()

,language_id,name,last_update
0,1,English,2006-02-15 05:02:19
1,2,Italian,2006-02-15 05:02:19
2,3,Japanese,2006-02-15 05:02:19
3,4,Mandarin,2006-02-15 05:02:19
4,5,French,2006-02-15 05:02:19


In [214]:
language.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   language_id  6 non-null      int64 
 1   name         6 non-null      object
 2   last_update  6 non-null      object
dtypes: int64(1), object(2)
memory usage: 276.0+ bytes


In [215]:
language.duplicated().unique()

array([False])

Limpieza tabla old_HDD

In [216]:
old_HDD.head()

,first_name,last_name,title,release_year,category_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14


In [217]:
old_HDD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   first_name    1000 non-null   object
 1   last_name     1000 non-null   object
 2   title         1000 non-null   object
 3   release_year  1000 non-null   int64 
 4   category_id   1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [218]:
unique_cols(old_HDD)

title           614
first_name       38
last_name        37
category_id      16
release_year      1
dtype: int64

In [219]:
old_HDD= old_HDD.drop("release_year", axis=1)

In [220]:
old_HDD.duplicated().unique()

array([False])

In [221]:
old_HDD.loc[old_HDD[['first_name', 'last_name', 'title']].duplicated(keep=False)] #por si acaso hay duplicados con este subset de columnas

,first_name,last_name,title,category_id


In [222]:
old_HDD['full_name']=old_HDD.first_name +' '+ old_HDD.last_name

Limpieza tabal "Rental"

In [223]:
rental.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [224]:
rental= rental.drop('last_update', axis=1)

In [225]:
rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rental_id     1000 non-null   int64 
 1   rental_date   1000 non-null   object
 2   inventory_id  1000 non-null   int64 
 3   customer_id   1000 non-null   int64 
 4   return_date   1000 non-null   object
 5   staff_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [226]:
unique_cols(rental)

rental_id       1000
inventory_id    1000
rental_date      999
return_date      997
customer_id      485
staff_id           2
dtype: int64

In [227]:
rental.duplicated().unique()

array([False])

Probando merges

In [228]:
'''count = 0
for title in old_HDD['title']:
    if title in film['title'].values:
        count += 1
print(count)'''

"count = 0\nfor title in old_HDD['title']:\n    if title in film['title'].values:\n        count += 1\nprint(count)"

In [229]:
'''count2=0
for name in old_HDD['full_name']:
    if name in actor['full_name'].values:
        count += 1
print(count)'''

"count2=0\nfor name in old_HDD['full_name']:\n    if name in actor['full_name'].values:\n        count += 1\nprint(count)"

In [230]:
title_catid = pd.merge(film, old_HDD[['title', 'category_id']], on='title', how='left')
film['category_id'] = title_catid['category_id']

In [231]:
film.head()

,film_id,title,description,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,category_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",6.0
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,1,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",6.0
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,1,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",6.0
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,1,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",6.0
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,1,6,2.99,130,22.99,G,Deleted Scenes,11.0


In [232]:
nancols(film) #no todas tenian esta información de old_hdd. Hay que remplazar estos nulos por 17 y indicar en la tabla de cat una nueva cat 17="unknown"

category_id    275
dtype: int64

In [233]:
film['category_id']=film['category_id'].fillna(17)

In [234]:
film['category_id']=film['category_id'].astype(int)

In [243]:
new_row = pd.DataFrame({'category_id': [17], 'name': ['unknown']})
category = pd.concat([category, new_row]).reset_index(drop=True)

In [235]:
old_HDD.head() 
''' 

Esta tabla la vamos a usar como conectora entre la tabla de peliculas y la tabla de actores. 

Por lo tanto nos interesa: 
 - vincular los IDs de la tabla actores a los full name de tabla old_HDD
 - vincular los IDS de las peliculas a los titulos de tabla old_HDD
 - crear una nueva columna 'film_actor_id' que cree un indice para las distintas combinaciones de title_id y actor_id (parece ser que esto se hará automaticamente no hace falta aqui)

 No nos interesa:
 - las columnas de first name, last name, title, category y full name. dejando unicamente: film_actor_id, film_id y actor_id

'''

" \n\nEsta tabla la vamos a usar como conectora entre la tabla de peliculas y la tabla de actores. \n\nPor lo tanto nos interesa: \n - vincular los IDs de la tabla actores a los full name de tabla old_HDD\n - vincular los IDS de las peliculas a los titulos de tabla old_HDD\n - crear una nueva columna 'film_actor_id' que cree un indice para las distintas combinaciones de title_id y actor_id (parece ser que esto se hará automaticamente no hace falta aqui)\n\n No nos interesa:\n - las columnas de first name, last name, title, category y full name. dejando unicamente: film_actor_id, film_id y actor_id\n\n"

In [236]:
merged = pd.merge(old_HDD, film[['film_id', 'title']], on='title', how='left')

In [237]:
merged.head()

,first_name,last_name,title,category_id,full_name,film_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,6,PENELOPE GUINESS,1
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2,PENELOPE GUINESS,23
2,PENELOPE,GUINESS,ANGELS LIFE,13,PENELOPE GUINESS,25
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,10,PENELOPE GUINESS,106
4,PENELOPE,GUINESS,CHEAPER CLYDE,14,PENELOPE GUINESS,140


In [238]:
merged = pd.merge(merged, actor[['actor_id', 'full_name']], on='full_name', how='left')

In [239]:
merged = merged.drop(['first_name', 'last_name', 'title', 'category_id', 'full_name'], axis=1)

In [246]:
actor=actor.drop(['first_name', 'last_name'] , axis=1)

In [253]:
language=language.drop(['last_update'],axis=1)

In [256]:
language = language.rename(columns={'name': 'language'}) 
#name es una palabra reservada en SQL...al querer hacer referencia a esta columna de la tabla, SQL no entiende

In [258]:
category = category.rename(columns={'name': 'category'})
#no hacia tanta falta en verdad. Puedes ponerlo entre comillas en SQL para que no lo asocie a la palabra reservada
#tambien puedes cambiar nombres de columna directamente desde workbench. learnings...

In [262]:
rental.loc[rental['inventory_id'].et]

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id


In [267]:
rental['inventory_id'].head

<bound method NDFrame.head of 0       367
1      1525
2      1711
3      2452
4      2079
5      2792
6      3995
7      2346
8      2580
9      1824
10     4443
11     1584
12     2294
13     2701
14     3049
15      389
16      830
17     3376
18     1941
19     3517
20      146
21      727
22     4441
23     3273
24     3961
25     4371
26     1225
27     4068
28      611
29     3744
30     4482
31     3832
32     1681
33     2613
34     1286
35     1308
36      403
37     2540
38     4466
39     2638
40     1761
41      380
42     2578
43     3098
44     1853
45     3318
46     2211
47     1780
48     2965
49     1983
50     1257
51     4017
52     1255
53     2787
54     1139
55     1352
56     3938
57     3050
58     2884
59      330
60     4210
61      261
62     4008
63       79
64     3552
65     1162
66      239
67     4029
68     3207
69     2168
70     2408
71     2260
72      517
73     1744
74     3393
75     3021
76     1303
77     4067
78     3299
79     2478
80     261

Exportar tablas


In [259]:

#actor.to_csv('../Data/actor_clean.csv', index=False) 
category.to_csv('../Data/category_clean.csv', index=False) 
#film.to_csv('../Data/film_clean.csv', index=False) 
#inventory.to_csv('../Data/inventory_clean.csv', index=False) 
language.to_csv('../Data/language_clean.csv', index=False) 
#old_HDD.to_csv('../Data/old_HDD_clean.csv', index=False) 
#rental.to_csv('../Data/rental_clean.csv', index=False) 
#merged.to_csv('../data/film_actor.csv', index = False)

